### Model Subclassing

In [8]:
from tensorflow import keras
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Concatenate, Dense
import numpy as np
from keras.utils import plot_model

### Subclassing the Model class

In [9]:
class CustomModel(Model):

    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = Concatenate()
        self.mixing_layer = Dense(64, activation="relu")
        self.priority_scorer = Dense(1, activation="sigmoid")
        self.department_classifier = Dense(num_departments, activation="softmax")

    def call(self, inputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)
        
        return priority, department

### Dummy data

In [10]:
vocabulary_size = 10000
num_tags = 100
num_samples = 1280
num_departments = 4

title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

inputs = {"title": title_data, "text_body": text_body_data, "tags": tags_data}

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

In [11]:
model = CustomerTicketModel(num_departments)
priority, department = model(inputs)

outputs = [priority_data, department_data]

In [12]:
model.compile(optimizer="rmsprop", loss=["mean_squared_error", "categorical_crossentropy"], 
              metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit(inputs, outputs, epochs=1)
model.evaluate(inputs, outputs)

priority_preds, department_preds = model.predict(inputs)

40/40 [==============================] - 0s 2ms/step
